In [1]:
import pickle

with open("data/MicroLens_1M_x1/feature_processor.pkl", "rb") as f:
    feature_processor = pickle.load(f)
print(feature_processor)

In [7]:
import pandas as pd
data_name = "item_info_ljh"
new_emb = pd.read_parquet(f"data/MicroLens_1M_x1/{data_name}.parquet")
item_feature = pd.read_parquet("dataset/MicroLens_1M_MMCTR/item_feature.parquet")[["item_id", "likes_level",  "views_level"]]

print(item_feature.head())

   item_id  likes_level  views_level
0        1            7            2
1        2            5            9
2        3            2            2
3        4            2            1
4        5            7            5


In [8]:
all_emb = pd.merge(new_emb, item_feature, on="item_id", how="left")

In [9]:
all_emb = all_emb.rename(columns={"likes_level": "likes", "views_level": "views"})
# all_emb.fillna(0, inplace=True)
print(all_emb.head())

   item_id        item_tags  \
0        0  [0, 0, 0, 0, 0]   
1        1  [0, 0, 0, 0, 1]   
2        2  [0, 0, 2, 3, 4]   
3        3  [0, 0, 5, 6, 7]   
4        4  [0, 0, 0, 8, 9]   

                                       item_emb_d128  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [-0.587724506855011, -0.38462838530540466, 0.4...   
2  [-2.5054404735565186, 1.5605803728103638, 0.23...   
3  [-1.0391175746917725, 1.031670093536377, -0.45...   
4  [0.1128326952457428, -0.7956982254981995, 0.70...   

                                             dis_emb  likes  views  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...    NaN    NaN  
1  [35, 9, 41, 18, 34, 1, 58, 8, 53, 8, 50, 49, 9...    7.0    2.0  
2  [35, 23, 6, 5, 27, 5, 32, 19, 23, 4, 16, 42, 2...    5.0    9.0  
3  [21, 9, 41, 18, 34, 55, 6, 37, 43, 4, 10, 4, 4...    2.0    2.0  
4  [46, 9, 6, 11, 22, 48, 6, 15, 24, 20, 22, 33, ...    2.0    1.0  


In [10]:
all_emb.to_parquet(f"data/MicroLens_1M_x1/{data_name}.parquet")

In [13]:
import pandas as pd
data = pd.read_parquet("dataset/MicroLens_1M_MMCTR/item_seq.parquet")


In [16]:
data.head()

,user_id,item_seq
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
from tqdm import tqdm
lens = []
for i, item in tqdm(data.iterrows(), total=len(data)):
    one_item = item["item_seq"]
    while(one_item[0] == 0):
        one_item = one_item[1:]
    lens.append(len(one_item))

data["len"] = lens
data["len"].describe()  

100%|██████████| 6000000/6000000 [07:50<00:00, 12746.39it/s]


count    6.000000e+06
mean     7.973784e+00
std      7.234855e+00
min      3.000000e+00
25%      4.000000e+00
50%      6.000000e+00
75%      9.000000e+00
max      1.000000e+02
Name: len, dtype: float64

In [1]:
# add gpt embedding
import pandas as pd
raw_data = pd.read_parquet("dataset/MicroLens_1M_MMCTR/item_feature.parquet")
print(raw_data.head())

   item_id                                         item_title     item_tags  \
0        1  Leian #TheLastKnightAmmishu #OriginalOutsideTh...           [1]   
1        2  If the heart is gone, use a stone to make it p...     [2, 3, 4]   
2        3  "I Lead the Empress to Battle the Three Realms...     [5, 6, 7]   
3        4        It's all superfluous! #Naruto #NarutoMobile        [8, 9]   
4        5  #NoHeartbeatChallenge made me drool a little #...  [10, 11, 12]   

   likes_level  views_level  \
0            7            2   
1            5            9   
2            2            2   
3            2            1   
4            7            5   

                                        txt_emb_BERT  \
0  [-0.35627136, 0.29088444, -0.3601495, -0.26140...   
1  [-0.32983607, 0.3043856, -0.13676094, -0.01752...   
2  [-0.25158748, 0.369733, -0.0891795, -0.1076934...   
3  [-0.4750408, 0.1370452, -0.3940854, -0.3176355...   
4  [-0.52664506, 0.38321885, -0.4115216, -0.35488...   

 

In [5]:
import torch
import pandas as pd
item_info = pd.read_parquet("data/MicroLens_1M_x1/item_info.parquet")
clip_disc_data = torch.load("data/indices_clip.pt")
gpt_disc_data = torch.load("data/indices_gpt.pt")
dis_emb = torch.cat([clip_disc_data, gpt_disc_data], dim=1)
print(dis_emb.shape)
new_data = pd.DataFrame()
new_data["item_id"] = raw_data["item_id"]
new_data["dis_emb"] = dis_emb.tolist()
new_data = pd.merge(item_info, new_data, on="item_id", how="left")
new_data.at[0, "dis_emb"]=[0 for i in range(len(dis_emb[0]))]
print(new_data.head())


torch.Size([91717, 1024])
   item_id                                            dis_emb
0        1  [35, 9, 41, 18, 34, 1, 58, 8, 53, 8, 50, 49, 9...
1        2  [35, 23, 6, 5, 27, 5, 32, 19, 23, 4, 16, 42, 2...
2        3  [21, 9, 41, 18, 34, 55, 6, 37, 43, 4, 10, 4, 4...
3        4  [46, 9, 6, 11, 22, 48, 6, 15, 24, 20, 22, 33, ...
4        5  [11, 9, 6, 61, 22, 6, 10, 8, 61, 15, 60, 4, 61...
   item_id        item_tags  \
0        0  [0, 0, 0, 0, 0]   
1        1  [0, 0, 0, 0, 1]   
2        2  [0, 0, 2, 3, 4]   
3        3  [0, 0, 5, 6, 7]   
4        4  [0, 0, 0, 8, 9]   

                                       item_emb_d128  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [-0.587724506855011, -0.38462838530540466, 0.4...   
2  [-2.5054404735565186, 1.5605803728103638, 0.23...   
3  [-1.0391175746917725, 1.031670093536377, -0.45...   
4  [0.1128326952457428, -0.7956982254981995, 0.70...   

                                             dis_emb  
0  [0, 0, 0, 0, 0, 0, 0

In [6]:
new_data.to_parquet("data/MicroLens_1M_x1/item_info_ljh.parquet")